In [1]:
import pandas as pd
import numpy as np
import sklearn
import os
from tqdm import tqdm, trange, tqdm_notebook
import re
import nltk
from multiprocessing import cpu_count, Pool
from functools import partial
import ipywidgets
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from rewrite.scorer import score_4class
tqdm.pandas()

In [2]:
X_train = pd.read_hdf("X_train_full_allfeatures-NOLABEL.h5", key="df")
y_train = pd.read_hdf("y_train_full.h5", key="df")

X_test = pd.read_hdf("X_TEST_full_allfeatures-NOLABEL.h5", key="df")
y_test = pd.read_hdf("y_TEST_full.h5", key="df")

print(X_train.shape)
print(X_test.shape)

(49972, 49)
(25413, 49)


In [3]:
X_train.head()

,Headline,articleBody,___clean_headline,___clean_headline_tokenized_lemmas,___clean_body,___clean_body_tokenized_lemmas,bin_count,bin_count_early,bin_count_stopless,word_overlap_features,...,chargram_2_first_hits,chargram_8_hits,chargram_8_early_hits,chargram_8_first_hits,chargram_4_hits,chargram_4_early_hits,chargram_4_first_hits,chargram_16_hits,chargram_16_early_hits,chargram_16_first_hits
0,Police find mass graves with at least '15 bodi...,Danny Boyle is directing the untitled film\n\n...,police find mass graves with at least 15 bodie...,"[police, find, mass, graf, with, at, least, 15...",danny boyle is directing the untitled film set...,"[danny, boyle, is, directing, the, untitled, f...",2,0,0,0.014085,...,0,0,0,0,0,0,0,0,0,0
1,Hundreds of Palestinians flee floods in Gaza a...,Hundreds of Palestinians were evacuated from t...,hundreds of palestinians flee floods in gaza a...,"[hundred, of, palestinian, flee, flood, in, ga...",hundreds of palestinians were evacuated from t...,"[hundred, of, palestinian, were, evacuated, fr...",10,7,7,0.046083,...,3,0,0,0,0,0,0,0,0,0
2,"Christian Bale passes on role of Steve Jobs, a...",30-year-old Moscow resident was hospitalized w...,christian bale passes on role of steve jobs ac...,"[christian, bale, pass, on, role, of, steve, j...",30 year old moscow resident was hospitalized w...,"[30, year, old, moscow, resident, wa, hospital...",5,4,1,0.030303,...,1,0,0,0,0,0,0,0,0,0
3,HBO and Apple in Talks for $15/Month Apple TV ...,(Reuters) - A Canadian soldier was shot at the...,hbo and apple in talks for 15 month apple tv s...,"[hbo, and, apple, in, talk, for, 15, month, ap...",reuters a canadian soldier was shot at the can...,"[reuters, a, canadian, soldier, wa, shot, at, ...",3,3,0,0.028169,...,1,0,0,0,0,0,0,0,0,0
4,Spider burrowed through tourist's stomach and ...,"Fear not arachnophobes, the story of Bunbury's...",spider burrowed through tourist s stomach and ...,"[spider, burrowed, through, tourist, s, stomac...",fear not arachnophobes the story of bunbury s ...,"[fear, not, arachnophobes, the, story, of, bun...",9,5,4,0.032727,...,4,0,0,0,0,0,0,0,0,0


In [4]:
X_test.head()

,Headline,articleBody,___clean_headline,___clean_headline_tokenized_lemmas,___clean_body,___clean_body_tokenized_lemmas,bin_count,bin_count_early,bin_count_stopless,word_overlap_features,...,chargram_2_first_hits,chargram_8_hits,chargram_8_early_hits,chargram_8_first_hits,chargram_4_hits,chargram_4_early_hits,chargram_4_first_hits,chargram_16_hits,chargram_16_early_hits,chargram_16_first_hits
0,Ferguson riots: Pregnant woman loses eye after...,A RESPECTED senior French police officer inves...,ferguson riots pregnant woman loses eye after ...,"[ferguson, riot, pregnant, woman, loses, eye, ...",a respected senior french police officer inves...,"[a, respected, senior, french, police, officer...",3,1,1,0.005405,...,3,0,0,0,0,0,0,0,0,0
1,Crazy Conservatives Are Sure a Gitmo Detainee ...,Dave Morin's social networking company Path is...,crazy conservatives are sure a gitmo detainee ...,"[crazy, conservative, are, sure, a, gitmo, det...",dave morin s social networking company path is...,"[dave, morin, s, social, networking, company, ...",1,1,0,0.006711,...,1,0,0,0,0,0,0,0,0,0
2,A Russian Guy Says His Justin Bieber Ringtone ...,A bereaved Afghan mother took revenge on the T...,a russian guy says his justin bieber ringtone ...,"[a, russian, guy, say, his, justin, bieber, ri...",a bereaved afghan mother took revenge on the t...,"[a, bereaved, afghan, mother, took, revenge, o...",4,2,1,0.015000,...,1,0,0,0,0,0,0,0,0,0
3,"Zombie Cat: Buried Kitty Believed Dead, Meows ...",Hewlett-Packard is officially splitting in two...,zombie cat buried kitty believed dead meows ba...,"[zombie, cat, buried, kitty, believed, dead, m...",hewlett packard is officially splitting in two...,"[hewlett, packard, is, officially, splitting, ...",1,1,0,0.003597,...,0,0,0,0,0,0,0,0,0,0
4,Argentina's President Adopts Boy to End Werewo...,An airline passenger headed to Dallas was remo...,argentina s president adopts boy to end werewo...,"[argentina, s, president, adopts, boy, to, end...",an airline passenger headed to dallas was remo...,"[an, airline, passenger, headed, to, dallas, w...",3,2,2,0.013158,...,2,0,0,0,0,0,0,0,0,0


In [5]:
def del_str_cols(df): # df should be X, e.g. X_train or X_dev
    del df["articleBody"]
    del df["Headline"]
    for col_name in df.columns:
        if "___" == col_name[0:3]:
            del df[col_name]
#del_str_cols()

In [6]:
#Xy_train = pd.concat([X_train, y_train], axis=1)
#print(Xy_train.shape)

In [7]:
#Xy_train.head()

In [8]:
#Xy_train = Xy_train[Xy_train["Stance"] != "unrelated"]

In [9]:
#print(Xy_train.shape)

In [10]:
#y_train = Xy_train["Stance"]
#X_train = Xy_train.drop("Stance", axis=1)
#print(X_train.shape)
#print(y_train.shape)

In [11]:
## START RANDOM FEATURE -- NUM OF CHARS IN HL AND BODY

In [12]:
def add_num_chars_feature(df, col_name):
    df["n_chars_"+col_name] = df.apply(lambda row: len(row[col_name]), axis=1)
print(X_train.shape)
print(X_test.shape)
add_num_chars_feature(X_train, "Headline")
add_num_chars_feature(X_train, "articleBody")
add_num_chars_feature(X_test, "Headline")
add_num_chars_feature(X_test, "articleBody")
print("--")
print(X_train.shape)
print(X_test.shape)

(49972, 49)
(25413, 49)
--
(49972, 51)
(25413, 51)


In [13]:
del_str_cols(X_train)
del_str_cols(X_test)

In [14]:
print(X_train.shape)
print(y_train.shape)

(49972, 45)
(49972,)


In [15]:
X_train, X_dev, y_train, y_dev = sklearn.model_selection.train_test_split(X_train, y_train, test_size=0.1, random_state=42, shuffle=True)

In [16]:
print(X_train.shape)
print(y_train.shape)
print(X_dev.shape)
print(y_dev.shape)

(44974, 45)
(44974,)
(4998, 45)
(4998,)


In [17]:
X_train.head()

,bin_count,bin_count_early,bin_count_stopless,word_overlap_features,wrf_hl_fake,wrf_hl_fraud,wrf_hl_hoax,wrf_hl_false,wrf_hl_deny,wrf_hl_denies,...,chargram_8_early_hits,chargram_8_first_hits,chargram_4_hits,chargram_4_early_hits,chargram_4_first_hits,chargram_16_hits,chargram_16_early_hits,chargram_16_first_hits,n_chars_Headline,n_chars_articleBody
44610,9,8,4,0.057377,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,76,1104
10499,4,3,1,0.023256,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,56,1712
43957,8,5,4,0.024465,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,144,3602
49559,4,2,1,0.009926,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,62,5394
1851,1,1,0,0.008403,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,48,949


In [18]:
clf = GradientBoostingClassifier(n_estimators=200, random_state=14128, verbose=True)

In [19]:
clf.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1       43999.6352           46.65s
         2       39373.4908           45.16s
         3       35623.4708           44.35s
         4       32556.4718           43.40s
         5       30020.4298           43.06s
         6       27905.8857           42.58s
         7       26123.1520           43.98s
         8       24611.2408           44.44s
         9       23329.1166           43.76s
        10       22192.1395           43.48s
        20       16722.0316           39.14s
        30       14984.2763           37.56s
        40       14274.3506           34.95s
        50       13881.4351           32.13s
        60       13602.9361           29.44s
        70       13382.1896           26.98s
        80       13196.2576           24.69s
        90       13010.8276           22.43s
       100       12873.6500           20.22s
       200       11758.9050            0.00s


GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=200,
              presort='auto', random_state=14128, subsample=1.0,
              verbose=True, warm_start=False)

In [20]:
preds = clf.predict(X_dev)
print(classification_report(y_dev, preds))
score, max_score = score_4class(y_dev, preds)
print("Weighted accuracy: "+str(score/max_score)+" ("+str(score)+" out of "+str(max_score)+")")

             precision    recall  f1-score   support

      agree       0.58      0.19      0.29       331
   disagree       0.60      0.03      0.06       102
    discuss       0.67      0.86      0.76       876
  unrelated       0.97      0.98      0.97      3689

avg / total       0.88      0.89      0.87      4998

Weighted accuracy: 0.8159103641456583 (1820.5 out of 2231.25)


In [21]:
preds = clf.predict(X_test)
print(classification_report(y_test, preds))
score, max_score = score_4class(y_test, preds)
print("Weighted accuracy: "+str(score/max_score)+" ("+str(score)+" out of "+str(max_score)+")")

             precision    recall  f1-score   support

      agree       0.36      0.10      0.16      1903
   disagree       0.10      0.00      0.01       697
    discuss       0.62      0.81      0.70      4464
  unrelated       0.95      0.98      0.96     18349

avg / total       0.82      0.86      0.83     25413

Weighted accuracy: 0.7617423023280764 (8875.25 out of 11651.25)


In [ ]:
crf = sklearn.ensemble.RandomForestClassifier(n_estimators=400, n_jobs=8, random_state=42)
crf.fit(X_train, y_train)

In [ ]:
preds = crf.predict(X_dev)
print(classification_report(y_dev, preds))

In [ ]:
score, max_score = score_4class(y_dev, preds)
print("Weighted accuracy: "+str(score/max_score)+" ("+str(score)+" out of "+str(max_score)+")")

In [ ]:
-----

In [ ]:
y_train.value_counts(normalize=True)

In [ ]:
crf = sklearn.ensemble.RandomForestClassifier(n_estimators=200, n_jobs=8, random_state=42)

In [ ]:
crf.fit(X_train, y_train)

In [ ]:
preds = crf.predict(X_dev)
print(classification_report(y_dev, preds))

In [ ]:
y_train.value_counts(normalize=True)

In [ ]:
# 4-class problem:
# By guessing most common category (unrelated), we would achieve approx. 73% accuracy. The baseline model achieves 88%.